In [1]:
# you should have intake-esm and intake-xarray installed
import intake
import xarray as xr
import numpy as np
import math as mt

In [2]:
import intake
cat = intake.open_catalog("../catalog.yaml")

In [3]:
run = cat.FESOM['tco2559-ng5']

In [4]:
data = run.original_3d.to_dask()

In [5]:
data

,Array,Chunk
Bytes,5.16 TiB,28.24 MiB
Shape,"(2776, 69, 7402886)","(1, 1, 7402886)"
Count,383101 Tasks,191544 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,5.16 TiB,28.24 MiB
Shape,"(2776, 69, 7402886)","(1, 1, 7402886)"
Count,383101 Tasks,191544 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [6]:
grid_elements = run.elem_grid.to_dask()

In [7]:
grid_elements

<xarray.Dataset>
Dimensions:          (grid_size: 14741520, grid_corners: 3, grid_rank: 1,
                      ncells: 7402886, nlinks_max: 8)
Dimensions without coordinates: grid_size, grid_corners, grid_rank, ncells,
                                nlinks_max
Data variables:
    grid_center_lon  (grid_size) float64 ...
    grid_corner_lon  (grid_size, grid_corners) float64 ...
    grid_center_lat  (grid_size) float64 ...
    grid_corner_lat  (grid_size, grid_corners) float64 ...
    grid_dims        (grid_rank) float64 ...
    grid_imask       (grid_size) float64 ...
    cell_area        (grid_size) float64 ...
    node_node_links  (ncells, nlinks_max) float64 ...
    triag_nodes      (grid_size, grid_corners) float64 ...
    coast            (grid_size) float64 ...
Attributes:
    Conventions:  CF-1.4
    history:      2022-03-19 08:26:26 GMT; Grid description file generated wi...

In [8]:
def scalar_g2r(al, be, ga, lon, lat):
    """
    Converts geographical coordinates to rotated coordinates.
    Parameters
    ----------
    al : float
        alpha Euler angle
    be : float
        beta Euler angle
    ga : float
        gamma Euler angle
    lon : array
        1d array of longitudes in geographical coordinates
    lat : array
        1d array of latitudes in geographical coordinates
    Returns
    -------
    rlon : array
        1d array of longitudes in rotated coordinates
    rlat : array
        1d araay of latitudes in rotated coordinates
    """

    rad = mt.pi / 180
    al = al * rad
    be = be * rad
    ga = ga * rad

    rotate_matrix = np.zeros(shape=(3, 3))

    rotate_matrix[0, 0] = np.cos(ga) * np.cos(al) - np.sin(ga) * np.cos(be) * np.sin(al)
    rotate_matrix[0, 1] = np.cos(ga) * np.sin(al) + np.sin(ga) * np.cos(be) * np.cos(al)
    rotate_matrix[0, 2] = np.sin(ga) * np.sin(be)
    rotate_matrix[1, 0] = -np.sin(ga) * np.cos(al) - np.cos(ga) * np.cos(be) * np.sin(
        al
    )
    rotate_matrix[1, 1] = -np.sin(ga) * np.sin(al) + np.cos(ga) * np.cos(be) * np.cos(
        al
    )
    rotate_matrix[1, 2] = np.cos(ga) * np.sin(be)
    rotate_matrix[2, 0] = np.sin(be) * np.sin(al)
    rotate_matrix[2, 1] = -np.sin(be) * np.cos(al)
    rotate_matrix[2, 2] = np.cos(be)
    
    #rotate_matrix = np.linalg.pinv(rotate_matrix)
    
    lat = lat * rad
    lon = lon * rad

    # geographical Cartesian coordinates:
    xr = np.cos(lat) * np.cos(lon)
    yr = np.cos(lat) * np.sin(lon)
    zr = np.sin(lat)

    # rotated Cartesian coordinates:
    xg = rotate_matrix[0, 0] * xr + rotate_matrix[0, 1] * yr + rotate_matrix[0, 2] * zr
    yg = rotate_matrix[1, 0] * xr + rotate_matrix[1, 1] * yr + rotate_matrix[1, 2] * zr
    zg = rotate_matrix[2, 0] * xr + rotate_matrix[2, 1] * yr + rotate_matrix[2, 2] * zr

    # rotated coordinates:
    rlat = np.arcsin(zg)
    rlon = np.arctan2(yg, xg)

    a = np.where((np.abs(xg) + np.abs(yg)) == 0)
    if a:
        lon[a] = 0

    rlat = rlat / rad
    rlon = rlon / rad

    return (rlon, rlat)

In [9]:
def vec_rotate_r2g(al, be, ga, lon, lat, urot, vrot, flag):
    """
    Rotate vectors from rotated coordinates to geographical coordinates.
    Parameters
    ----------
    al : float
        alpha Euler angle
    be : float
        beta Euler angle
    ga : float
        gamma Euler angle
    lon : array
        1d array of longitudes in rotated or geographical coordinates (see flag parameter)
    lat : array
        1d array of latitudes in rotated or geographical coordinates (see flag parameter)
    urot : array
        1d array of u component of the vector in rotated coordinates
    vrot : array
        1d array of v component of the vector in rotated coordinates
    flag : 1 or 0
        flag=1  - lon,lat are in geographical coordinate
        flag=0  - lon,lat are in rotated coordinate
    Returns
    -------
    u : array
        1d array of u component of the vector in geographical coordinates
    v : array
        1d array of v component of the vector in geographical coordinates
    """

    #   first get another coordinate
    if flag == 1:
        (rlon, rlat) = scalar_g2r(al, be, ga, lon, lat)
    else:
        rlon = lon
        rlat = lat
        (lon, lat) = scalar_r2g(al, be, ga, rlon, rlat)

    #   then proceed...
    rad = mt.pi / 180
    al = al * rad
    be = be * rad
    ga = ga * rad

    rotate_matrix = np.zeros(shape=(3, 3))
    rotate_matrix[0, 0] = np.cos(ga) * np.cos(al) - np.sin(ga) * np.cos(be) * np.sin(al)
    rotate_matrix[0, 1] = np.cos(ga) * np.sin(al) + np.sin(ga) * np.cos(be) * np.cos(al)
    rotate_matrix[0, 2] = np.sin(ga) * np.sin(be)
    rotate_matrix[1, 0] = -np.sin(ga) * np.cos(al) - np.cos(ga) * np.cos(be) * np.sin(
        al
    )
    rotate_matrix[1, 1] = -np.sin(ga) * np.sin(al) + np.cos(ga) * np.cos(be) * np.cos(
        al
    )
    rotate_matrix[1, 2] = np.cos(ga) * np.sin(be)
    rotate_matrix[2, 0] = np.sin(be) * np.sin(al)
    rotate_matrix[2, 1] = -np.sin(be) * np.cos(al)
    rotate_matrix[2, 2] = np.cos(be)

    rotate_matrix = np.linalg.pinv(rotate_matrix) 
    
    rlat = rlat * rad
    rlon = rlon * rad
    lat = lat * rad
    lon = lon * rad

    #   vector in rotated Cartesian
    txg = -vrot * np.sin(rlat) * np.cos(rlon) - urot * np.sin(rlon)
    tyg = -vrot * np.sin(rlat) * np.sin(rlon) + urot * np.cos(rlon)
    tzg = vrot * np.cos(rlat)

    #   vector in geo Cartesian
    txr = (
        rotate_matrix[0, 0] * txg
        + rotate_matrix[0, 1] * tyg
        + rotate_matrix[0, 2] * tzg
    )
    tyr = (
        rotate_matrix[1, 0] * txg
        + rotate_matrix[1, 1] * tyg
        + rotate_matrix[1, 2] * tzg
    )
    tzr = (
        rotate_matrix[2, 0] * txg
        + rotate_matrix[2, 1] * tyg
        + rotate_matrix[2, 2] * tzg
    )

    #   vector in geo coordinate
    v = (
        -np.sin(lat) * np.cos(lon) * txr
        - np.sin(lat) * np.sin(lon) * tyr
        + np.cos(lat) * tzr
    )
    u = -np.sin(lon) * txr + np.cos(lon) * tyr

    u = np.array(u)
    v = np.array(v)

    return (u, v)

In [10]:
data.u

,Array,Chunk
Bytes,10.27 TiB,56.23 MiB
Shape,"(2776, 69, 14741520)","(1, 1, 14741520)"
Count,383101 Tasks,191544 Chunks
Type,float32,numpy.ndarray


In [11]:
u = data.u[0,0,:].values
v = data.v[0,0,:].values

In [12]:
lon = grid_elements.grid_center_lon.values
lat = grid_elements.grid_center_lat.values

In [13]:
u_rot, v_rot = vec_rotate_r2g(
        50, 15, -90, lon, lat, u, v, flag=1
    )

In [14]:
u

array([-0.07756004, -0.0940954 , -0.16058405, ...,  0.06806916,
        0.00160212,  0.03429421], dtype=float32)

In [15]:
v

array([-0.08295829, -0.05170697, -0.13513608, ..., -0.07524757,
       -0.3716321 , -0.29633322], dtype=float32)

In [16]:
u_rot

array([-0.0887712 , -0.05881184, -0.14761075, ..., -0.07166887,
       -0.3713527 , -0.29512383])

In [17]:
v_rot

array([ 0.07083299,  0.08982607,  0.14919807, ..., -0.07182744,
       -0.01449679, -0.04349018])